<a href="https://colab.research.google.com/github/A1DS19/Movie-review-classifier-BIRNN/blob/master/Transfer_learning_IMDB_GloVe_embeddings.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import tensorflow as tf
import numpy as np
import pandas as pd
from tensorflow import keras
import tensorflow_datasets as tfds
!nvidia-smi -L

GPU 0: Tesla P100-PCIE-16GB (UUID: GPU-ae37bb4a-067f-47df-1eda-28006f3312c5)


In [2]:
train_data, ds_info = tfds.load('imdb_reviews',
                       split='train',
                       as_supervised=True,
                       with_info=True)

test_data = tfds.load('imdb_reviews',
                       split='test',
                       as_supervised=True)

Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

Generating splits...:   0%|          | 0/3 [00:00<?, ? splits/s]

Generating train examples...:   0%|          | 0/25000 [00:00<?, ? examples/s]

Shuffling ~/tensorflow_datasets/imdb_reviews/plain_text/1.0.0.incompleteU59OIE/imdb_reviews-train.tfrecord*...…

Generating test examples...:   0%|          | 0/25000 [00:00<?, ? examples/s]

Shuffling ~/tensorflow_datasets/imdb_reviews/plain_text/1.0.0.incompleteU59OIE/imdb_reviews-test.tfrecord*...:…

Generating unsupervised examples...:   0%|          | 0/50000 [00:00<?, ? examples/s]

Shuffling ~/tensorflow_datasets/imdb_reviews/plain_text/1.0.0.incompleteU59OIE/imdb_reviews-unsupervised.tfrec…

Dataset imdb_reviews downloaded and prepared to ~/tensorflow_datasets/imdb_reviews/plain_text/1.0.0. Subsequent calls will reuse this data.


In [3]:
train_data = train_data.batch(32).prefetch(tf.data.AUTOTUNE)
test_data = test_data.batch(32).prefetch(tf.data.AUTOTUNE)

In [4]:
ds_info

tfds.core.DatasetInfo(
    name='imdb_reviews',
    full_name='imdb_reviews/plain_text/1.0.0',
    description="""
    Large Movie Review Dataset.
    This is a dataset for binary sentiment classification containing substantially more data than previous benchmark datasets. We provide a set of 25,000 highly polar movie reviews for training, and 25,000 for testing. There is additional unlabeled data for use as well.
    """,
    config_description="""
    Plain text
    """,
    homepage='http://ai.stanford.edu/~amaas/data/sentiment/',
    data_path='~/tensorflow_datasets/imdb_reviews/plain_text/1.0.0',
    file_format=tfrecord,
    download_size=80.23 MiB,
    dataset_size=129.83 MiB,
    features=FeaturesDict({
        'label': ClassLabel(shape=(), dtype=tf.int64, num_classes=2),
        'text': Text(shape=(), dtype=tf.string),
    }),
    supervised_keys=('text', 'label'),
    disable_shuffling=False,
    splits={
        'test': <SplitInfo num_examples=25000, num_shards=1>,
        '

In [5]:
for example in train_data.take(1):
    print(example)

(<tf.Tensor: shape=(32,), dtype=string, numpy=
array([b"This was an absolutely terrible movie. Don't be lured in by Christopher Walken or Michael Ironside. Both are great actors, but this must simply be their worst role in history. Even their great acting could not redeem this movie's ridiculous storyline. This movie is an early nineties US propaganda piece. The most pathetic scenes were those when the Columbian rebels were making their cases for revolutions. Maria Conchita Alonso appeared phony, and her pseudo-love affair with Walken was nothing but a pathetic emotional plug in a movie that was devoid of any real meaning. I am disappointed that there are movies like this, ruining actor's like Christopher Walken's good name. I could barely sit through it.",
       b'I have been known to fall asleep during films, but this is usually due to a combination of things including, really tired, being warm and comfortable on the sette and having just eaten a lot. However on this occasion I fell

In [6]:
train_sentences = train_data.map(lambda sentence, labels:sentence)
train_labels = train_data.map(lambda sentence, labels:labels)

test_sentences = test_data.map(lambda sentence, labels:sentence)
test_labels = test_data.map(lambda sentence, labels:labels)

In [7]:
MAX_LENGTH = 600
MAX_TOKENS = 20000

text_vectorization = keras.layers.TextVectorization(max_tokens=MAX_TOKENS,
                                                    standardize='lower',
                                                    output_sequence_length=MAX_TOKENS)
text_vectorization.adapt(train_sentences)

### Load GloVe embeddings

In [8]:
%%bash
wget https://nlp.stanford.edu/data/glove.6B.zip
unzip glove.6B.zip

Archive:  glove.6B.zip
  inflating: glove.6B.50d.txt        
  inflating: glove.6B.100d.txt       
  inflating: glove.6B.200d.txt       
  inflating: glove.6B.300d.txt       


--2022-09-29 05:43:09--  https://nlp.stanford.edu/data/glove.6B.zip
Resolving nlp.stanford.edu (nlp.stanford.edu)... 171.64.67.140
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://downloads.cs.stanford.edu/nlp/data/glove.6B.zip [following]
--2022-09-29 05:43:09--  https://downloads.cs.stanford.edu/nlp/data/glove.6B.zip
Resolving downloads.cs.stanford.edu (downloads.cs.stanford.edu)... 171.64.64.22
Connecting to downloads.cs.stanford.edu (downloads.cs.stanford.edu)|171.64.64.22|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 862182613 (822M) [application/zip]
Saving to: ‘glove.6B.zip’

     0K .......... .......... .......... .......... ..........  0% 2.53M 5m25s
    50K .......... .......... .......... .......... ..........  0% 2.66M 5m17s
   100K .......... .......... .......... .......... ..........  0% 77.0M 3m35s
   150K .......... .......... .

In [9]:
path_to_glove_file = './glove.6B.100d.txt'

embeddings_index = {}
with open(path_to_glove_file) as f:
    for line in f:
        word, coefs = line.split(maxsplit=1)
        coefs = np.fromstring(coefs, 'f', sep=' ')
        embeddings_index[word] = coefs
        
print(f'Found {len(embeddings_index)} word vectors')

Found 400000 word vectors


In [10]:
EMBEDDING_DIM = 100
vocabulary = text_vectorization.get_vocabulary()
word_index = dict(zip(vocabulary, range(len(vocabulary))))
embedding_matrix = np.zeros((MAX_TOKENS, EMBEDDING_DIM))

for word, i in word_index.items():
    if i < MAX_TOKENS:
        embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector

In [11]:
from tensorflow.keras import layers

def create_model(RNN_UNITS, LAYERS, TRAIN_EMB):
    inputs = layers.Input(shape=(1,), dtype=tf.string)
    x = text_vectorization(inputs)
    x = keras.layers.Embedding(MAX_TOKENS,
                                         EMBEDDING_DIM,
                                         embeddings_initializer=keras.initializers.Constant(embedding_matrix),
                                         trainable=TRAIN_EMB,
                                         mask_zero=True)(x)
    x = layers.Bidirectional(layers.LSTM(RNN_UNITS, return_sequences=True))(x)
    x = layers.Dropout(0.5)(x)
    x = layers.Bidirectional(layers.LSTM(RNN_UNITS*2))(x)
    x = layers.Dropout(0.5)(x)
    outputs = layers.Dense(1, activation='sigmoid')(x)
    return keras.Model(inputs, outputs)

In [12]:
model = create_model(32, 4, False)

In [13]:
model.compile(loss=keras.losses.binary_crossentropy,
              optimizer=keras.optimizers.Adam(),
              metrics=['accuracy', 'Precision', 'Recall'])

In [14]:
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 1)]               0         
                                                                 
 text_vectorization (TextVec  (None, 20000)            0         
 torization)                                                     
                                                                 
 embedding (Embedding)       (None, 20000, 100)        2000000   
                                                                 
 bidirectional (Bidirectiona  (None, 20000, 64)        34048     
 l)                                                              
                                                                 
 dropout (Dropout)           (None, 20000, 64)         0         
                                                                 
 bidirectional_1 (Bidirectio  (None, 128)              66048 

In [15]:
history_model = model.fit(train_data,
                          epochs=10)

Epoch 1/10
782/782 [==============================] - 164s 186ms/step - loss: 0.5442 - accuracy: 0.7221 - precision: 0.7240 - recall: 0.7180
Epoch 2/10
782/782 [==============================] - 146s 187ms/step - loss: 0.4961 - accuracy: 0.7659 - precision: 0.7530 - recall: 0.7915
Epoch 3/10
782/782 [==============================] - 145s 186ms/step - loss: 0.4388 - accuracy: 0.8012 - precision: 0.7939 - recall: 0.8137
Epoch 4/10
782/782 [==============================] - 147s 188ms/step - loss: 0.3969 - accuracy: 0.8258 - precision: 0.8261 - recall: 0.8252
Epoch 5/10
782/782 [==============================] - 147s 188ms/step - loss: 0.3660 - accuracy: 0.8404 - precision: 0.8390 - recall: 0.8424
Epoch 6/10
782/782 [==============================] - 147s 188ms/step - loss: 0.3455 - accuracy: 0.8523 - precision: 0.8511 - recall: 0.8540
Epoch 7/10
782/782 [==============================] - 147s 188ms/step - loss: 0.3267 - accuracy: 0.8629 - precision: 0.8600 - recall: 0.8670
Epoch 8/10
78

### Train model with fine tuning on pretrained embedding layer

In [16]:
model_ft = create_model(32, 4, True)

In [17]:
model_ft.compile(loss=keras.losses.binary_crossentropy,
              optimizer=keras.optimizers.Adam(),
              metrics=['accuracy', 'Precision', 'Recall'])

In [18]:
history_model_ft = model.fit(train_data,
                          epochs=10)

Epoch 1/10
782/782 [==============================] - 147s 188ms/step - loss: 0.2728 - accuracy: 0.8874 - precision: 0.8849 - recall: 0.8906
Epoch 2/10
782/782 [==============================] - 146s 187ms/step - loss: 0.2603 - accuracy: 0.8944 - precision: 0.8919 - recall: 0.8976
Epoch 3/10
782/782 [==============================] - 147s 187ms/step - loss: 0.2412 - accuracy: 0.9032 - precision: 0.9031 - recall: 0.9032
Epoch 4/10
782/782 [==============================] - 146s 186ms/step - loss: 0.2303 - accuracy: 0.9089 - precision: 0.9084 - recall: 0.9095
Epoch 5/10
782/782 [==============================] - 146s 186ms/step - loss: 0.2213 - accuracy: 0.9131 - precision: 0.9124 - recall: 0.9140
Epoch 6/10
782/782 [==============================] - 146s 187ms/step - loss: 0.2103 - accuracy: 0.9191 - precision: 0.9203 - recall: 0.9177
Epoch 7/10
782/782 [==============================] - 146s 187ms/step - loss: 0.1990 - accuracy: 0.9239 - precision: 0.9238 - recall: 0.9241
Epoch 8/10
78